Segmenting and Clustering Neighborhoods in Toronto


Importa all necessary libraries


In [2]:
!pip install beautifulsoup4

In [14]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=abe1259aa830460cd4046ce65f4d9058b0247abe54a1911ad1fcf042899d80d0
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [27]:
!pip install requests

In [0]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import wikipedia as wp
import json

In [45]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
bs = BeautifulSoup(url.text, 'lxml')
table = bs.find('table')
td = table.find_all('td')
postcode = []
borough = []
neighbourhood = []

for i in range(0, len(td), 3):
    postcode.append(td[i].text.strip())
    borough.append(td[i+1].text.strip())
    neighbourhood.append(td[i+2].text.strip())
            
dataframe = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
dataframe.columns = ['Postal Code', 'Borough', 'Neighborhood'] ## The dataframe will have 3 columns
dataframe['Borough'].replace('Not assigned', np.nan, inplace=True)
dataframe.dropna(subset=['Borough'], inplace=True) #ignore unassigned Boroughs
dataframe['Neighborhood'].replace('Not assigned', dataframe['Borough'], inplace=True) ##Empty Neighberhoods take the name of the Borough
dataframe = dataframe.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply('/ '.join).reset_index() ##Multiple Neighborhoods in on PC are combined
dataframe.columns = ['Postal Code', 'Borough', 'Neighborhood']
dataframe.shape

(103, 3)

In [47]:
url="http://cocl.us/Geospatial_data"
geo = pd.read_csv(url)
geo.columns = ['Postal Code', 'Latitude', 'Longitude']
toronto_df = pd.merge(dataframe, geo, on=['Postal Code'], how='inner')
toronto_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.688905,-79.554724
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437


**We will work only with boroughs containing the word Toronto**


In [50]:
toronto_data = toronto_df[toronto_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,Moore Park / Summerhill East,43.689574,-79.383160
9,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686412,-79.400049
